# Module 2 Summative Lab

## Introduction

For today's section, we're going to work on a single big lab to apply everything we've learned in Module 2!

## About This Lab

A quick note before getting started--this lab isn't like other labs you seen so far. This lab is meant to take ~8 hours to complete, so it's much longer and more challenging that the average labs you've seen so far. If you feel like this lab is challenging or that you might be struggling a bit, don't fret--that's by design! With everything we've learned about Web Scraping, APIs, and Databases, the best way to test our knowledge of it is to build something substantial! 

## The Project

In this lab, we're going to make use of everything we've learned about APIs, databases, and Object-Oriented Programming to **_Extract, Transform, and Load_** (or **_ETL_**, for short) some data from a SQL database into a MongoDB Database. 

You'll find a database containing information about soccer teams and the matches they've played in the file `database.sqlite`. For this project, our goal is to get the data we think is important from this SQL database, do some calculations and data transformation, and then store everything in a MongoDB database. 

Let's get into the specifics of this project.

### The Goal

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this [kaggle page](https://www.kaggle.com/laudanum/footballdelphi).  Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database. 

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
* The team's win percentage on days where it was raining during games in the 2011 season. 

#### Getting the Weather Data

Note that for this last calculation, you'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the [DarkSky API](https://darksky.net/dev) to get the historical weather data for that day. Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely german, so go ahead and just use the weather in Berlin, Germany as a proxy for this information. If it was raining in Berlin on the day the game was played, count that as rain game--**_you do not need to try and figure out the actual weather at each game's location, because we don't have that information!_**

#### NOTE: The DarkSky API is limited to 1000 free API calls a day, so be sure to test your model on very small samples. Otherwise, you'll hit the rate limit!

## Project Architecture

Unlike previous labs, this lab is more open-ended, and will require you to make design decisions and plan out your strategy for building a system with this many working parts. However, **_using Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well structured classes in your solution!_** Although it may seem easier to "just start coding", this is a classic beginner's mistake. Instead, think about separating out the different functionalities you'll need to reach your goal, and then build classes to handle each. For instance, at minimum, you'll need to:

* Query the SQL database
* Calculate summary statistics
* Get the weather data from the DarkSky API
* Load the data into MongoDB

We **_strongly recommend_** you consider creating separate classes for handling at least some of these of these tasks.  Be sure to plan the inputs, outputs, and methods for each class before you begin coding! 

**_NOTE:_** We have provided some empty classes below. You are welcome to delete them and use a different architecture for this project if you so choose.  You do not have to use each of them, they are just there to give you an idea of what you could sorts of classes you may want to consider using.

### Rapid Prototyping and Refactoring

It's totally okay to try to get a task working without using OOP. For instance, when experimenting with the DarkSky API for getting historical weather data, it makes sense to just write the code in the cells and rapidly iterate until you get it all working. However, once you get it working, you're not done--you should then **_Refactor_** your code into functions or classes to make your code more modular, reusable, understandable, and maintainable! 

In short--do what you need to do to get each separate piece of functionality working, and then refactor it into a class after you've figured it out!

### Some Final Advice

You haven't built anything this big or complex thus far, so you may not yet fully realize how much trial and error goes into it. If your code keeps breaking, resist the urge to get frustrated, and just keep working. Software development is an iterative process!  No one writes perfect code that works the first time for something this involved. You're going to run into _a lot_ of small errors in this project, right up until the point where it just works, and then you're done! However, you can reduce these errors by planning out your code, and thinking about how all of the pieces fit together before you begin coding. Once you have some basic understanding of how it all will work, then you'll know what you need to build, and then all that is left is to build it!

In short:

* Plan ahead--you'll thank yourself later!
* Errors and broken code aren't bad, they're normal. 
* Keep working, and stay confident--you can do this!

Good luck--we look forward to seeing your completed project!

# Our lab

### Importing necessary packages

In [1]:
import os
import numpy
import PIL
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup
import requests
import json
import apikey as api
import pymongo

### Connecting our database for use in python

In [ ]:
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

### Moving the data into Pandas so that we can calculate summary statistics and make visualizations
After reviewing the different tables in the kaggle database, we found that we could get all the information we needed was contained in the Matches table.

In [ ]:
c.execute("""SELECT * FROM Matches WHERE Season = 2011""")
matches = pd.DataFrame(c.fetchall())
matches.columns = [x[0] for x in c.description]

### We started by creating a MatchInfo Class
This class contained the functions necessary to find total goals, total games won, and total games lost by the teams.

In [ ]:
class MatchInfo:
    
    def __init__(self, df):
        self.df = df
        
    def total_goals(self):
        
        """Takes in our dataframe in question and outputs a dataframe 
        with the total number of goals for each team."""
        
        home_goals = self.df.groupby('HomeTeam').sum()[['FTHG']]
        away_goals = self.df.groupby('AwayTeam').sum()[['FTAG']]
        total = home_goals.join(away_goals, how = 'inner')
        total['totals'] = total.FTHG + total.FTAG
        total_goals = total[['totals']] 
        total_goals.reset_index(inplace = True)
        total_goals.columns = ['TeamName', 'total_goals_scored']
        return total_goals
    
    def team_names(self):
        
        """Takes in our dataframe in question and 
        outputs a list of all our team names."""
        
        return list(self.df['HomeTeam'].unique())

    def total_won(self):
        
        """Takes in our dataframe in question and outputs a 
        dataframe of the total games won per team."""
        
        self.df['home_won'] = self.df.FTR.apply(lambda x: 1 if x == 'H' else 0)
        self.df['away_won'] = self.df.FTR.apply(lambda x: 1 if x == 'A' else 0)
        home_won = self.df.groupby('HomeTeam').sum()[['home_won']]
        away_won = self.df.groupby('AwayTeam').sum()[['away_won']]
        total_won = home_won.join(away_won, how = 'inner')
        total_won['total_won'] = total_won.home_won + total_won.away_won
        total_won = total_won[['total_won']]
        total_won.reset_index(inplace = True)
        total_won.columns = ['TeamName', 'total_games_won']
        return total_won
    
    def total_lost(self):
        
        """Takes in our dataframe in question and outputs a 
        dataframe of the total games lost per team."""
        
        self.df['home_lost'] = self.df.FTR.apply(lambda x: 1 if x == 'A' else 0)
        self.df['away_lost'] = self.df.FTR.apply(lambda x: 1 if x == 'H' else 0)
        home_lost = self.df.groupby('HomeTeam').sum()[['home_lost']]
        away_lost = self.df.groupby('AwayTeam').sum()[['away_lost']]
        total_lost = home_lost.join(away_lost, how = 'inner')
        total_lost['total_lost'] = total_lost.home_lost + total_lost.away_lost
        total_lost = total_lost[['total_lost']]
        total_lost.reset_index(inplace = True)
        total_lost.columns = ['TeamName', 'total_games_lost']
        return total_lost
    
    
match_info = MatchInfo(matches)
# Instantiating the dataframe to use when calling functions from our class

### Transforming dataframe for visualization
Here we transformed our dataframe to a dictionary to be able to loop through and plot individual graphs of wins and losses per team

In [ ]:
lost = match_info.total_lost()
won = match_info.total_won()
won_lost = won.merge(lost, how = 'inner', on = 'TeamName')
lost['type'] = 'lost'
lost.rename(columns = {'total_games_lost': 'points'}, inplace = True)
won['type'] = 'won'
won.rename(columns = {'total_games_won': 'points'}, inplace = True)
concat = pd.concat([won, lost], axis = 0)
concat.sort_values(by = 'TeamName', inplace = True)
concat.set_index(['TeamName', 'type'], inplace = True)
unstacked = concat.unstack()
unstacked_dict = unstacked.to_dict('index')

### We created a class to build and directly store graphs as images into a directory
To more easily move our data into MongoDB, we directly stored our images into a new directory.

In [ ]:
class BuildGraph:
    
    def __init__(self, dictionary):
        self.dictionary = dictionary

    def build_graph(self):
        
        """This function takes in a dictionary and outputs a graph for each 
        team as well as stats on how many wins and losses the team had in the 2011 season. 
        It also stores each graph as an individual image to the new directory 'Team_Graphs'
        """
        os.makedirs('Team_Graphs')
        for team in self.dictionary:
            plt.bar(x = [0, 1], height = [self.dictionary[team]['points', 'lost'], 
                              self.dictionary[team]['points', 'won']], tick_label = ['Won', 'Lost'])
            plt.title(team)
            plt.xlabel('Outcomes')
            plt.ylabel('Number of games')
            plt.savefig("Team_Graphs/{}.png".format(team))
            plt.close()
            

to_graph = BuildGraph(unstacked_dict)

### Transforming our images into a dictionary of arrays
To be able to store our images into MongoDB, we needed to turn our images of our graphs into arrays. The following function uses Python Imaging Library (PIL) to open each image and numpy to change the format to array.

In [ ]:
figs = os.listdir('Team_Graphs')

fig_dictionary = {}
for fig in figs:
    img = PIL.Image.open('team_graphs/{}'.format(fig)). convert('L')
    array = numpy.array(img)
    fig_dictionary.update({fig: array})

### Transforming the dataframe for use with the Dark Sky API

To be able to use the dark sky api, we found it easier to convert our dates to unix time.  We created a class to convert our dates.  Then we created a class to fetch the weather data for our dates and return a dictionary of with the weather matched to matchID.

In [ ]:
class weather_prepare:
    def __init__(self,data):
        self.data = data

    def prepare_dataframe_for_weather(self):
        
        """"We set our date and time to Unix timecode so as to 
            more easily access the date information from DarkSky""""
        
        data.set_index('Match_ID', inplace = True)
        data['Date'] = pd.to_datetime(data['Date'])
        data['Unix_Date'] = (data['Date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

unix = weather_prepare(data)

unix.prepare_dataframe_for_weather()

dates = data['Unix_Date']


class weather_scrape:
    def __init__(self):
        self.url = 'https://api.darksky.net/forecast'
        self.key = api.API_Key4
        self.latitude = "52.5200"
        self.longitude = "13.4050"
        self.exclude = 'currently,flags,minutely,hourly,alerts'
        self.df = data
        self.dates = dates

    def api_call(self):
        """"Using our API, we are getting weather condition information for each date 
            and storing this in a dictionary for future use
        """"
        weather_dict = {}
        for date in self.dates:
            good_url = "{}/{}/{},{},{}?exclude={}".format(self.url,self.key,self.latitude, 
                                                          self.longitude, date, self.exclude)   
            response = requests.get(good_url)
            print(response)
            weather_data = response.json()['daily']['data'][0]['icon']
            weather_dict.update({date: weather_data})
        return weather_dict

weather_data = weather_scrape()

weather = weather_data.api_call()

We then created a fucntion to take the weather dictionary and convert it into a dataframe to be joined with the team information.

In [ ]:
def adding_weather_to_df(weather):
    """"This function takes in our dictionary of weather information and 
        outputs a new dataframe with the weather conditions which is then 
        merged to the main dataframe
    """"
    wdf = pd.DataFrame(weather, index = ['weather'])
    wdf2 = pd.DataFrame.transpose(wdf)
    wdf2 = wdf2.reset_index()
    wdf2.rename(columns={'index': 'Unix_Date'}, inplace=True)
    dfw = pd.merge(data, wdf2, how = 'left', on = 'Unix_Date')
    return dfw

We then transformed the weather dataframe to find out the percentage of games played when it rained.

In [ ]:
final = adding_weather_to_df(weather)
final['weather'] = final.weather.apply(lambda x: 1 if x == 'rain' else 0)
final['all_weather'] = final.weather.apply(lambda x: 1)

# We created 2 new columns: 1 that allows us to sum the number of times it rained and 1 that allows us to sum 
# the total number of games

home_rain = final.groupby('HomeTeam').sum()[['weather']]
home_all = final.groupby('HomeTeam').sum()[['all_weather']]
away_rain = final.groupby('AwayTeam').sum()[['weather']]
away_all = final.groupby('AwayTeam').sum()[['all_weather']]

# We then summed these values to find the proportion of days when it rained

total_rain = home_rain.weather + away_rain.weather
total_weather = home_all.all_weather +away_all.all_weather
avg_weather = total_rain/total_weather*100

# We found the average number of times it rained

total_wins = match_info.total_won()
total_goal = match_info.total_goals()
games = total_wins.merge(total_goal, how = 'inner', on = 'TeamName')
dw = pd.DataFrame(avg_weather)
dw.reset_index(inplace = True)
dw.rename(columns = {0: 'rain_percent'}, inplace = True)
FINAL = pd.concat([games, dw], axis = 1)
FINAL.drop('HomeTeam', axis = 1, inplace = True)

# We merged our dataframes so as to see all of our data together and provide for easier MongoDB insertion

### Inserting data into MongoDB
We have not got to test our class to insert our data into the mongodb but this was our approach to do that eventually.

In [ ]:
class Mongo_maker:
    
    def __init__(self):
        self.myclient = pymongo.MongoClient('mongodb://localhost:27017')
        self.mydb = self.myclient['2011_season_stats']
        self.mycollection = self.mydb['2011_season_stats']
    
    def format_data(self, name, totalgoals, totalwins, bar_graph, rain_wins):
        data = {"team_name": name,
               "total_goals": totalgoals,
               "total_wins": totalwins,
               "win_loss_graph": bar_graph,
               "rain_wins": rain_wins}
        
        return data
    
    def insert_record(self, record):
        return self.mycollection.insert_one(data)

# Summary

In this lab, we dug deep and used everything we've learned so far about python programming, databases, HTTP requests and API calls to ETL data from a SQL database into a MongoDB instance!